<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных-из-файла,-предобработка" data-toc-modified-id="Изучение-данных-из-файла,-предобработка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файла, предобработка</a></span></li><li><span><a href="#Разбиение-данных-на-обучающую,-валидационную-и-тестовую-выборки" data-toc-modified-id="Разбиение-данных-на-обучающую,-валидационную-и-тестовую-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбиение данных на обучающую, валидационную и тестовую выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-вменяемость" data-toc-modified-id="Проверка-модели-на-вменяемость-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на вменяемость</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Рекомендация тарифов сотовой связи

Оператор сотовой связи «Мегалайн» хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Для анализа имеются данные о поведении клиентов, которые уже перешли на эти тарифы. 


В проекте нужно построить модель для задачи классификации, которая выберет подходящий тариф. После построения модели следует проверить её на тестовой выборке и получить максимально большое значение метрики *accuracy* (не менее 0,75).

## Изучение данных из файла, предобработка

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 

Загрузим данные из файла и ознакомимся с таблицей.

In [2]:
try:
    main_data = pd.read_csv('users_behavior.csv')
except:
    main_data = pd.read_csv('/datasets/users_behavior.csv')
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Видно, что в таблице отсутствуют пропуски, имена столбцов корректные. Посмотрим теперь сами данные.

In [3]:
main_data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Столбцы calls и messages имеют вещественный тип данных, а должны иметь целочисленный тип.  Выполним далее преобразование.

Столбцы minutes и mb_used могут хранить данные с дробными значениями (к примеру, абонент проговорил 311,9 минут), но это не имеет практического смысла. Мобильный оператор всё равно будет округлять минуты в большую сторону. Дробные значения мегабайтов интернет-трафика тоже не актуальны. Поэтому эти два столбца должны тоже иметь целочисленный тип. Выполним далее преобразование с округлением в большую сторону.

In [4]:
main_data['calls'] = main_data['calls'].astype('int')
main_data['messages'] = main_data['messages'].astype('int')
main_data['minutes'] = np.ceil(main_data['minutes']).astype('int')
main_data['mb_used'] = np.ceil(main_data['mb_used']).astype('int')

In [5]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   calls     3214 non-null   int32
 1   minutes   3214 non-null   int32
 2   messages  3214 non-null   int32
 3   mb_used   3214 non-null   int32
 4   is_ultra  3214 non-null   int64
dtypes: int32(4), int64(1)
memory usage: 75.5 KB


Преобразования типов столбцов прошли корректно. Исследуем теперь баланс классов в столбце с целевым признаком.

In [6]:
main_data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Видно, что в столбце целевых признаков более чем в два раза больше значений False, чем True. Это следует учесть при подготовке выборок в дальнейшем.

## Разбиение данных на обучающую, валидационную и тестовую выборки

Так как для исследований нет отдельной спрятанной тестовой выборки, то придётся исходный набор данных делить на три выборки: обучающую, валидационную и тестовую в соотношении 60%:20%:20%. Для получения таких выборок используем функцию train_test_split, вызвав её два раза. При этом при каждом вызове функции будем использовать стратификацию по целевым признакам, так как они распределены неравномерно в исходной таблице.

In [7]:
temp_data, test_data = train_test_split(
    main_data,test_size=0.2, stratify=main_data['is_ultra'], random_state=123)
train_data, valid_data = train_test_split(
    temp_data,test_size = 0.25, stratify=temp_data['is_ultra'], random_state=123)

In [8]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(1928, 5)
(643, 5)
(643, 5)


Разбиение данных на три выборки прошло успешно. Теперь выделим из выборок обычные и целевые признаки.

In [9]:
train_features = train_data.drop(columns=['is_ultra'])
train_target = train_data['is_ultra']
valid_features = valid_data.drop(columns=['is_ultra'])
valid_target = valid_data['is_ultra']
test_features = test_data.drop(columns=['is_ultra'])
test_target = test_data['is_ultra']

Проверим как распределены целевые признаки во всех выборках.

In [10]:
print('Обучающая выборка:\n', train_target.value_counts(), '\n')
print('Валидационная выборка:\n', valid_target.value_counts(), '\n')
print('Тестовая выборка:\n', test_target.value_counts())

Обучающая выборка:
 0    1337
1     591
Name: is_ultra, dtype: int64 

Валидационная выборка:
 0    446
1    197
Name: is_ultra, dtype: int64 

Тестовая выборка:
 0    446
1    197
Name: is_ultra, dtype: int64


Распределение целевых признаков в разных выборках выглядит схожим.

## Исследование моделей

Будем исследовать три вида моделей: дерево решений, случайный лес и логистическую регрессию. При этом в первых двух видах моделей попробуем подобрать оптимальные гиперпараметры, проверяя качество моделей на валидационной выборке. 
В алгоритмы обучения моделей добавим взвешивание классов (аргумент class_weight). Для ускорения обучения моделей "Случайный лес" и логистической регрессии применим гиперпараметр n_jobs=-1, что означает использование всех ядер и потоков процессора.

Начнём с модели "Дерево решений". Будем изменять значения гиперпараметра max_depth.

In [11]:
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, class_weight='balanced', random_state=123)
    model.fit(train_features, train_target)
    result = model.score(valid_features, valid_target)
    if result > best_result:
        best_depth = depth
        best_result = result

print(f'Лучшая модель дерева решений имеет глубину {best_depth}, значение accuracy={best_result:.3f}')

Лучшая модель дерева решений имеет глубину 5, значение accuracy=0.788


Теперь исследуем модель "Случайный лес". Будем изменять значения гиперпараметров n_estimators, max_depth и criterion.

In [12]:
best_result = 0
best_est = 0
best_depth = 0
best_crit= ''
for est in range(5, 101, 5):
    for depth in range (1, 21):
        for crit in ['gini', 'entropy']:
            model = RandomForestClassifier(n_estimators=est, max_depth=depth, criterion=crit, class_weight='balanced',
                                            n_jobs=-1, random_state=123) 
            model.fit(train_features, train_target) 
            result = model.score(valid_features, valid_target) 
            if result > best_result:
                best_est = est
                best_depth = depth
                best_crit = crit
                best_result = result

print(f'Лучшая модель случайного леса имеет количество деревьев {best_est}, глубину {best_depth}, критерий {best_crit}, ' 
      f'значение accuracy={best_result:.3f}')

Лучшая модель случайного леса имеет количество деревьев 30, глубину 10, критерий entropy, значение accuracy=0.806


Напоследок, исследуем модель логистической регрессии.

In [13]:
model = LogisticRegression(max_iter=200, random_state=123, n_jobs=-1) 
model.fit(train_features, train_target) 
result = model.score(valid_features, valid_target)
print(f'Модель логистической регрессии имеет значение accuracy={result:.3f}')

Модель логистической регрессии имеет значение accuracy=0.753


**Вывод:** наилучший результат (самое большое значение accuracy) показала модель случайного леса RandomForestClassifier с настроенными гиперпараметрами n_estimators, max_depth и criterion.

## Проверка модели на тестовой выборке

Используем модель со значениями гиперпараметров, которые показали наилучший результат при исследовании на валидационной выборке. Но теперь проверять работу модели будем с использованием тестовой выборки.

In [14]:
best_model = RandomForestClassifier(n_estimators=best_est, max_depth=best_depth, criterion=best_crit, n_jobs=-1, random_state=123) 
best_model.fit(train_features, train_target) 
result = best_model.score(test_features, test_target)
print(f'Значение accuracy при проверке выбранной модели на тестовой выборке: {result:.3F}')

Значение accuracy при проверке выбранной модели на тестовой выборке: 0.802


Попробуем теперь улучшить результат и обучить выбранную модель на суммарной тренировочной и валидационной выборках. В этом случае у модели будет больше данных для обучения. Ранее, я делил исходный датасет на три выборки функцией train_test_split в два этапа. На первом этапе данные разделились на тестовую и временную выборку в соотношении 20% к 80%. Эта временная выборка по сути и есть суммарная тренировочная и валидационная. Сформируем из неё признаки и целевые признаки, а затем заново обучим модель.

In [15]:
train_valid_features = temp_data.drop(columns=['is_ultra'])
train_valid_target = temp_data['is_ultra']
best_model.fit(train_valid_features, train_valid_target)
result = best_model.score(test_features, test_target)
print(f'Значение accuracy при проверке на тестовой выборке заново обученной модели на суммарной ' 
      f'тренировочной и валидационной выборках : {result:.3F}')

Значение accuracy при проверке на тестовой выборке заново обученной модели на суммарной тренировочной и валидационной выборках : 0.801


Результат серьёзно не изменился.

**Вывод:** выбранная модель случайного леса успешно прошла проверку и показала значение accuracy выше 0.75. Попытки использовать для обучения модели суммарную обучающую и валидационную выборки значительного результата не принесли.

## Проверка модели на вменяемость

Для проверки выбранной модели на вменяемость сравним её качество с качеством модели константного предсказания - DummyClassifier. Эта модель будет предсказывать одинаковые значения, которые встречаются в тренировочном наборе целевых признаков наиболее часто. 

In [16]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=123)
dummy_model.fit(train_features, train_target)
result = dummy_model.score(test_features, test_target)
print(f'Значение accuracy при проверке константной модели на тестовой выборке: {result:.3F}')

Значение accuracy при проверке константной модели на тестовой выборке: 0.694


Как и ожидалось, значение accuracy у константной модели невысокое. У выбранной модели случайного леса оно существенно выше. Но сравним качество этих моделей, построив матрицу ошибок. 

Посмотрим ещё раз как распределены значения в тестовом наборе целевых признаков.

In [17]:
test_target.value_counts()

0    446
1    197
Name: is_ultra, dtype: int64

In [18]:
dummy_pred = dummy_model.predict(test_features)
confusion_matrix(test_target, dummy_pred, labels = [0, 1])

array([[446,   0],
       [197,   0]], dtype=int64)

Константная модель определила, что в наборе тренировочных целевых признаков преобладают значения "False". И сделала такие предсказания для тестовой выборки. Все значения "False" были автоматически угаданы, но при этом вышли ложные предсказания для "True".

Построим теперь матрицу ошибок для выбранной в проекте модели.

In [19]:
best_pred = best_model.predict(test_features)
tn, fp, fn, tp = confusion_matrix(test_target, best_pred, labels = [0, 1]).ravel()
print(f'Выбранная в проекте модель допустила {fn} ложноотрицательных ответов и {fp} ложноположительных')

Выбранная в проекте модель допустила 100 ложноотрицательных ответов и 28 ложноположительных


## Общий вывод

В проекте была произведена минимальная преобработка исходных данных и их разделение на обучающую, валидационную и тестовую выборки. При разделении на выборки была применена стратификация по целевому признаку, что в дальнейшем улучшило метрику качества моделей.

Далее в проекте были построены три модели машинного обучения: дерево решений, случайный лес, логистическая регрессия. Для моделей подбирались значения их гиперпараметров с целью улучшения качества и получения более высокого значения accuracy. 
Наилучшие результаты удалось получить в модели случайного леса с определёнными значениями гиперпараметров. 

Выбранная модель успешно прошла проверку на тестовой выборке и показала значение accuracy более 0,75. Также модель прошла проверку на адекватность в сравнении с константной моделью. Попытки использовать для обучения модели суммарную обучающую и валидационную выборки значительного результата не принесли.